In [1]:
# Import statements
import glob
from spectral_cube import SpectralCube

In [33]:
# Use dask for parallelization
dask = True

if dask:
    from dask.diagnostics import ProgressBar
    pbar = ProgressBar()
    pbar.register()

In [2]:
# Get the cube we're making a mask from
freq_spw = '273_spw51'
pb_fn = glob.glob(f"/blue/adamginsburg/abulatek/brick/symlinks/imaging_results/source_ab*{freq_spw}*clean*.pb")
cube_fn = glob.glob(f"/blue/adamginsburg/abulatek/brick/symlinks/imaging_results/source_ab*{freq_spw}*clean*.image") # We need the units of this... and no one knows why
pb = SpectralCube.read(pb_fn[0], format = 'casa_image')
cube = SpectralCube.read(cube_fn[0], format = 'casa_image')

/blue/adamginsburg/abulatek/anaconda/lib/python3.7/site-packages/casa_formats_io-0.1-py3.7-linux-x86_64.egg/casa_formats_io/casa_dask.py:232: RuntimeWarning: divide by zero encountered in long_scalars
  factors = [f for f in range(stacks[dim] + 1) if stacks[dim] % f == 0]
/blue/adamginsburg/abulatek/anaconda/lib/python3.7/site-packages/casa_formats_io-0.1-py3.7-linux-x86_64.egg/casa_formats_io/casa_dask.py:232: RuntimeWarning: divide by zero encountered in long_scalars
  factors = [f for f in range(stacks[dim] + 1) if stacks[dim] % f == 0]


In [32]:
# Mask the cube above a PB gain level of 0.8; ignore the unit, it's just there to make SpectralCube happy and has no interpretation
mask = pb[-1] > 0.8 * pb.unit

[########################################] | 100% Completed |  0.5s


In [35]:
# Stolen partially from makemask.py; mask cubes and extract spectra
import time
import os
from spectral_cube.dask_spectral_cube import DaskSpectralCube

os.chdir("/blue/adamginsburg/abulatek/brick/symlinks/imaging_results/")
print(os.getcwd())

for suffix in ("image.pbcor.fits", "image"):
    for fn in glob.glob(f"source_ab*{suffix}"):
        t0 = time.time()
        outf_ = fn.replace(suffix,"max_CROPPED.fits")
        outfn = f'spectra/{outf_}'
        print(fn, outfn)
        if not os.path.exists(outfn):
            if not dask:
                cube = SpectralCube.read(fn)
                print(cube)
                cube_masked = cube.with_mask(mask)
                mxspec = cube_masked.max(axis=(1,2), how='slice', progressbar=True)
                meanspec = cube_masked.mean(axis=(1,2), how='slice', progressbar=True)
                
            else:
                print(fn)
                cube = DaskSpectralCube.read(fn.replace(".fits", ""), format='casa_image')
                print(cube)
                print(cube.shape)
                print(mask.shape)
                cube_masked = cube.with_mask(mask)
                mxspec = cube_masked.max(axis=(1,2))
                meanspec = cube_masked.mean(axis=(1,2))

            mxspec.write(outfn, overwrite=True)
            meanspec.write(outfn.replace("max", "mean"))
            print(time.time() - t0)

/orange/adamginsburg/brick_alma_linesurvey/2019.1.00092.S/imaging_results
source_ab_102_spw29_dirty.image spectra/source_ab_102_spw29_dirty.max_CROPPED.fits
source_ab_102_spw29_dirty.image


/blue/adamginsburg/abulatek/anaconda/lib/python3.7/site-packages/casa_formats_io-0.1-py3.7-linux-x86_64.egg/casa_formats_io/casa_dask.py:232: RuntimeWarning: divide by zero encountered in long_scalars
  factors = [f for f in range(stacks[dim] + 1) if stacks[dim] % f == 0]


DaskVaryingResolutionSpectralCube with shape=(1920, 512, 512) and unit=Jy / beam and chunk size (80, 256, 512):
 n_x:    512  type_x: RA---SIN  unit_x: deg    range:   266.528130 deg:  266.560501 deg
 n_y:    512  type_y: DEC--SIN  unit_y: deg    range:   -28.719152 deg:  -28.690763 deg
 n_s:   1920  type_s: FREQ      unit_s: Hz     range: 101549443159.977 Hz:103423216948.801 Hz
(1920, 512, 512)
(512, 512)
[                                        ] | 0% Completed |  0.1s

[########################################] | 100% Completed | 10.2s
[########################################] | 100% Completed | 10.2s
[########################################] | 100% Completed | 11.5s
[########################################] | 100% Completed | 11.6s
27.69101071357727
source_ab_103_spw31_clean_2sigma_n50000_masked_3sigma_pbmask0p18.image spectra/source_ab_103_spw31_clean_2sigma_n50000_masked_3sigma_pbmask0p18.max_CROPPED.fits
source_ab_103_spw31_clean_2sigma_n50000_masked_3sigma_pbmask0p18.image


/blue/adamginsburg/abulatek/anaconda/lib/python3.7/site-packages/casa_formats_io-0.1-py3.7-linux-x86_64.egg/casa_formats_io/casa_dask.py:232: RuntimeWarning: divide by zero encountered in long_scalars
  factors = [f for f in range(stacks[dim] + 1) if stacks[dim] % f == 0]


...

DaskVaryingResolutionSpectralCube with shape=(3840, 512, 512) and unit=Jy / beam and chunk size (128, 256, 512):
 n_x:    512  type_x: RA---SIN  unit_x: deg    range:   266.528130 deg:  266.560501 deg
 n_y:    512  type_y: DEC--SIN  unit_y: deg    range:   -28.719152 deg:  -28.690763 deg
 n_s:   3840  type_s: FREQ      unit_s: Hz     range: 267901562851.931 Hz:269776044440.248 Hz
(3840, 512, 512)
(512, 512)
[                                        ] | 0% Completed |  0.1s

[########################################] | 100% Completed | 25.5s
[########################################] | 100% Completed | 25.6s
[########################################] | 100% Completed | 26.4s
[########################################] | 100% Completed | 26.5s
63.99330711364746
source_ab_134_spw45_dirty.image spectra/source_ab_134_spw45_dirty.max_CROPPED.fits
source_ab_134_spw45_dirty.image


/blue/adamginsburg/abulatek/anaconda/lib/python3.7/site-packages/casa_formats_io-0.1-py3.7-linux-x86_64.egg/casa_formats_io/casa_dask.py:232: RuntimeWarning: divide by zero encountered in long_scalars
  factors = [f for f in range(stacks[dim] + 1) if stacks[dim] % f == 0]


DaskVaryingResolutionSpectralCube with shape=(3840, 512, 512) and unit=Jy / beam and chunk size (128, 256, 512):
 n_x:    512  type_x: RA---SIN  unit_x: deg    range:   266.528130 deg:  266.560501 deg
 n_y:    512  type_y: DEC--SIN  unit_y: deg    range:   -28.719152 deg:  -28.690763 deg
 n_s:   3840  type_s: FREQ      unit_s: Hz     range: 131879076591.442 Hz:133753642785.728 Hz
(3840, 512, 512)
(512, 512)
[                                        ] | 0% Completed |  0.1s

[########################################] | 100% Completed | 15.5s
[########################################] | 100% Completed | 15.6s
[########################################] | 100% Completed | 14.7s
[########################################] | 100% Completed | 14.8s
41.99548149108887
